### 1. Imports, constants, etc.

In [1]:
import os
import pandas as pd

INPUT_DIR = "input"
INPUT_DIR_EVVET = os.path.join(INPUT_DIR, 'evvet')

OUTPUT_DIR = "output"
OUTPUT_EVVET_FILE = os.path.join(OUTPUT_DIR, 'evvet_master.csv')

OUTPUT_EVVET_META = os.path.join(OUTPUT_DIR, "evvet_meta.json")

### 2. Fetch, merge, clean, output.

In [2]:
import datetime 
import json
import datetime

def load_and_merge_evvet_data():
    current_datetime = datetime.datetime.now(datetime.timezone.utc)
    last_updated = current_datetime.strftime("%B %d, %Y %H:%M:%S")
    print(f"Last updated:\n{last_updated}")

    file_pattern = '.csv'

    # Delete the output file if it exists
    if os.path.exists(OUTPUT_EVVET_FILE):
        os.remove(OUTPUT_EVVET_FILE)

    # List all CSV files in the input directory
    csv_files = [f for f in os.listdir(INPUT_DIR_EVVET) if f.endswith(file_pattern)]
    
    df_list = []
    for file in csv_files:
        df = pd.read_csv(os.path.join(INPUT_DIR_EVVET, file))
        df.dropna(how='all', inplace=True)
        print(f"\nProcessing file: {file}...")
        print(f"Number of rows: {len(df)}")
        df_list.append(df)
    
    # Concatenate all dataframes into one
    master_df = pd.concat(df_list, ignore_index=True)
    master_df.drop_duplicates(inplace=True)

    # Reorder columns to move 'AER form' to the last index if it exists in the dataframe
    if 'AER form' in master_df.columns:
        cols = master_df.columns.tolist()
        cols.append(cols.pop(cols.index('AER form')))
        master_df = master_df[cols]

    # Move 'Drug' column to the end if it exists in the dataframe
    if 'Drug' in master_df.columns:
        cols = master_df.columns.tolist()
        cols.append(cols.pop(cols.index('Drug')))
        master_df = master_df[cols]

    # Move 'Received date' column to the first index if it exists in the dataframe
    if 'Received date' in master_df.columns:
        cols = master_df.columns.tolist()
        cols.insert(0, cols.pop(cols.index('Received date')))
        master_df = master_df[cols]

    # Sort the dataframe by 'Received date'
    master_df.sort_values(by='Received date', inplace=True)

    # Drop any empty rows in master_df
    master_df.dropna(how='all', inplace=True)

    # Change Received Date column's format
    master_df['Received date'] = pd.to_datetime(master_df['Received date'])

    # Check for any rows where dates couldn't be parsed
    if master_df['Received date'].isna().any():
        print("Some dates couldn't be parsed and were set to NaT")

    # Save the updated master dataframe
    master_df.to_csv(OUTPUT_EVVET_FILE, index=False)
    print("\nMaster file compiled!")

    print()

    print(f"Total cases: {len(master_df)}")
    print(f"Animals affected: {master_df['Animals affected'].sum()}")
    print(f"Animals treated: {master_df['Animals treated'].sum()}")
    print(f"Animals died: {master_df['Animals died'].sum()}")

    print()

    meta = {"last_updated": current_datetime.isoformat()} 
    with open(OUTPUT_EVVET_META, "w") as f:
        json.dump(meta, f)
    print(f'{OUTPUT_EVVET_META} updated!')

load_and_merge_evvet_data()


Last updated:
July 13, 2024 20:49:37

Processing file: 2021.csv...
Number of rows: 383

Processing file: 2023.csv...
Number of rows: 5133

Processing file: 2022.csv...
Number of rows: 3170

Processing file: 2024.csv...
Number of rows: 8293

Master file compiled!

Total cases: 16979
Animals affected: 17807
Animals treated: 20660.0
Animals died: 1910.0

output/evvet_meta.json updated!
